In [ ]:
from IPython.display import HTML

HTML('''<script>
hide_code=true; 
function code_toggle() {
   if (hide_code){
       $('div.input').hide();
   } else {
       $('div.input').show();
   }
   hide_code = !hide_code
} 
$( document ).ready(code_toggle);
</script>
NOTE: the code in this notebook is hidden for better readability.  
To toggle on/off, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
# importing BeautifulSoup from bs4 module 
from bs4 import BeautifulSoup 
# importing requests 
import requests 
 
### Code for initial page collection and to see html for page to help with 
### coding to parse page contents
### Preserved in case page formats change and existing code must be modified
# set URL for page 
#r = requests.get("https://secure.meetcontrol.com/divemeets/system/totaljudgesscoresext.php?meetnum=5363&eventnum=260&dvrnum=ALL&eventtype=9") 
#r = requests.get("https://secure.meetcontrol.com/divemeets/system/totaljudgesscoresext.php?meetnum=4186&eventnum=1030&dvrnum=ALL&eventtype=9")
# Look at html for the page
#print(r.content)
# Look at html for the page in a more readable format
#soup = BeautifulSoup(r.content, 'html5lib')
#print(soup.prettify())

In [ ]:
# import the library for parsing through the html
from html.parser import HTMLParser

# import csv for writing out saved data
import csv

# set flags for extracting the relevant data
data_start = 0      #flag that table containing data is following
meet_name = 0       #flag that next data entry is meet name
meet_date = 0       #flag that next data entry is meet date
meet_nm_found = 0   #flag that name for meet has been collected
meet_dt_found = 0   #flag that date for meet has been collected
diver = 0           #flag that next data entry is diver name
diver_found = 0     #flag that diver name has been collected and judges and scores are next
get_judge = 0       #flag that next data entry is judge name
judge_found = 0     #flag that judge name has been captures and to collect scores
get_score = 0       #flag for getting score
count_score = 0     #counting how many slots for scores have been parsed, max is 11
# Initialize the variables for saving the information for each line of data
Meet = ""
Date = ""
Event = ""
Diver = ""
Judge = ""
Scores = [None] * 11

#Define the functions for handling the tags and data
class MyHTMLParser(HTMLParser):
    
    #look at the start tag to tell us what is coming next and set appropriate flag
    def handle_starttag(self, tag, attrs):
        global data_start, get_judge, diver_found, get_score, count_score
        #If tag has table, the data we want to collect will be following
        if 'table' in tag:
            data_start = 1
        #If we are in data collection mode, check for tags in which we are interested
        if data_start == 1:
            #print("Encountered a start tag:", tag)
            #Judge's name follows an a tag
            if "a" in tag:
                if diver_found:
                    get_judge = 1
            #If we are collecting scores, count how many scores we have encountered
            if get_score:
                if "td" in tag:
                    count_score += 1

    #Look at the end tag to see if we are finished and reset appropriate flags
    def handle_endtag(self, tag):
        global data_start, meet_name, meet_date, meet_nm_found, meet_dt_found, diver, diver_found 
        global judge_found, get_score, get_judge, count_score, Scores
        
        #We don't care about end tags if we are not collecting the data
        if data_start == 1:
            #print("Encountered an end tag :", tag)
            
            #tr is the end of the row with judge's scores
            if "tr" in tag:
                get_score = 0
                
            #If we are at the end of a table, reset all flags for the next table
            if 'table' in tag:
                data_start = 0
                meet_name = 0
                meet_date = 0
                meet_nm_found = 0
                meet_dt_found = 0
                diver = 0
                diver_found = 0
                get_judge = 0
                judge_found = 0
                get_score = 0
                count_score = 0
                Scores = [None] * 11

    #Appropriately handle the data
    def handle_data(self, data):
        global data_start, meet_name, meet_date, meet_nm_found, meet_dt_found, diver, diver_found 
        global judge_found, get_score, get_judge, count_score, Meet, Date, Event, Diver, Judge, Scores
        
        #If we are collecting data, identify what was read and save that data, if wanted
        if data_start:
            #print("Encountered some data  :", data)
            
            # Save event name
            if (meet_nm_found & meet_dt_found):
                # Identify if it is an event - it will have boys or girls in the name
                if "Boys" in data:
                    #save off event
                    #print("Saving boy event  :", data)
                    Event = data
                    meet_nm_found = 0
                    meet_dt_found = 0 
                if "Girls" in data:
                    #save off event
                    #print("Saving girl event  :", data)
                    Event = data
                    meet_nm_found = 0
                    meet_dt_found = 0 
                    
            # Save meet name        
            if meet_name:
                #save off meet name
                #print("Saving meet name  :", data)
                Meet = data
                meet_nm_found = 1
                meet_name = 0
            
            if "Meet:" in data:
                #set flag to collect meet name from next data record
                meet_name = 1
                
            # Save meet date    
            if meet_date:
                #save off meet date
                #print("Saving meet date  :", data)
                Date = data
                meet_date = 0
                meet_dt_found = 1
                
            if "Date:" in data:
                #set flag to collect meet date from next data record
                meet_date = 1
                
            # If have diver name    
            if diver_found:
                
                # If have judge, save scores
                if judge_found & get_score:
                    
                    # If less than the maximum number of scores, save score
                    if count_score < 12:
                        #save scores 
                        #print("Saving score for dive # ",count_score, ", Score = ", data)
                        Scores[count_score-1] = data
                    # All scores saved, reset
                    else:
                        print("Writing data :", Meet, Date, Event, Diver, Judge,  
                              Scores[0], Scores[1], Scores[2], Scores[3], 
                              Scores[4], Scores[5], Scores[6], Scores[7],                               
                              Scores[8], Scores[9], Scores[10])
                        # append information to data file
                        writer = csv.writer(csvFile)
                        writer.writerow([Meet, Date, Event, Diver, Judge, 
                                             Scores[0], Scores[1], Scores[2], Scores[3], 
                                             Scores[4], Scores[5], Scores[6], Scores[7], 
                                             Scores[8], Scores[9], Scores[10]])
                            
                        #reset flags
                        count_score = 0
                        get_score = 0
                        
                # Save judge name        
                if get_judge:
                    # saving judge name
                    #print("Saving judge   :", data)
                    Judge = data
                    judge_found = 1
                    get_judge = 0
                    get_score = 1
                    Scores = [None] * 11
                    
            # Save diver name        
            if diver:
 
                # save off diver name
                # need to separate out the diver's club
                #print("Saving diver  :", data)
                Diver = data
                diver = 0
                diver_found = 1

            if "Diver:" in data:
                #set flag to collect diver name from next data record
                diver = 1

# Open file for saving data so that rows can be appended
with open('divemeets_data.csv', 'a') as csvFile:
    
    # Open the file containing the list of urls to be scraped
    with open('divemeets_urls') as f:
        # remove the carriage return from the end of the line
        url_list = f.read().splitlines()
        # find out how many urls are in the list
        num_url = len(url_list)
        print("number of url's = ", num_url)
        # set url retrieval index to zero 
        i = 0
        # while we haven't fetched all pages, keep going
        while i < num_url:
            # get the url from the list
            url = url_list[i]
            # display the path of the page being retrieved
            print(url)
            # get the page
            r = requests.get(url)
            # run the parser to extract the data
            parser = MyHTMLParser(  )
            # feed the page into the parser
            parser.feed(str(r.content))
            # increase the index to get the next path from the list
            i += 1

# close the data file, we are done
csvFile.close()
